https://python.langchain.com/docs/tutorials/rag/
</br>https://python.langchain.com/docs/integrations/text_embedding/
</br>https://platform.openai.com/settings/organization/billing/overview

In [62]:
from pathlib import Path

import pandas as pd
from jinja2 import Template

from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from settings import OPENAI_API_KEY

# Loading docs

In [55]:
PROMPT_TEMPLATE = Template(Path('/Users/caiopavesi/Code/bmw-job-matcher/data/prompt.jinja').read_text())
APPLICANT_DOCUMENTS_FILES_PATH = '/Users/caiopavesi/Library/Mobile Documents/com~apple~CloudDocs/0/Work/Job applications/Templates/Latest'

In [7]:
llm = ChatOpenAI(
    model = "gpt-4.1",
    temperature = 0,
    api_key = OPENAI_API_KEY
)

In [3]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")

In [4]:
vector_store = Chroma(
    collection_name = "example_collection",
    embedding_function = embeddings,
    persist_directory =  "../data/chroma",
)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

In [6]:
loader = DirectoryLoader(APPLICANT_DOCUMENTS_FILES_PATH, glob = "*.pdf", loader_cls = PyPDFLoader)
documents = loader.load()
all_splits = text_splitter.split_documents(documents)
document_ids = vector_store.add_documents(documents = all_splits)

# Creating the RAG